# Setup notebook

In [1]:
import geopandas as gp
import bw2regional as br
import bw2data as bd
import bw2calc as bc
from pathlib import Path
import numpy as np
from warnings import warn

In [2]:
MESSAGE = """"
Site-generic LCA will produce incorrect results with regionalized LCIA methods.

This was fixed in bw2calc 2.0.dev3.

See: https://github.com/brightway-lca/brightway2-calc/commit/485238e791aadf7764784a58494abcb33933ba4b"""

if bc.__version__ < (2, 0, "DEV3"):
    warn(MESSAGE)

# Setup and reset project

In [3]:
bd.projects.set_current("case-study")

In [4]:
for db_name in list(bd.databases):
    del bd.databases[db_name]
    
for meth in list(bd.methods):
    del bd.methods[meth]
    
for gc in list(br.geocollections):
    del br.geocollections[gc] 
    
for inter in list(br.intersections):
    del br.intersections[inter]     

# Setup database

In [5]:
db = bd.Database("case")

#declare the geolocation of the map of 1 single spatial delineation for LCI and lCIA
db.register(geocollections=["regions"])

# Setup geocollections

In [6]:
DATA_PATH = (Path.cwd().parent / "files").resolve()
DATA_PATH, list(DATA_PATH.iterdir())

(PosixPath('/Users/cmutel/Code/brightway/from-the-ground-up/files'),
 [PosixPath('/Users/cmutel/Code/brightway/from-the-ground-up/files/CF_polygones.gpkg'),
  PosixPath('/Users/cmutel/Code/brightway/from-the-ground-up/files/rice_provinces.gpkg')])

In [7]:
br.geocollections["watersheds"] = {
    'filepath': str(DATA_PATH / 'CF_polygones.gpkg'),
    # 'layer': 'CF_margina', 
    'field': 'ID',
}
br.geocollections["watersheds"] 

{'filepath': '/Users/cmutel/Code/brightway/from-the-ground-up/files/CF_polygones.gpkg',
 'field': 'ID',
 'sha256': 'e7f3c957ca998c4c90a03afe5b657cb55669f5515bc6e0f81aa917aa0ed565a2',
 'kind': 'vector'}

In [8]:
br.geocollections["regions"] = {
    'filepath': str(DATA_PATH / 'rice_provinces.gpkg'),
    'field': 'iso_3166_2',
}
br.geocollections["regions"] 

{'filepath': '/Users/cmutel/Code/brightway/from-the-ground-up/files/rice_provinces.gpkg',
 'field': 'iso_3166_2',
 'sha256': '1e2aab70520e4cd748a8324f278511234ae57a5375a7fdf33c002c5d1f1a0f2f',
 'kind': 'vector'}

# Setup regionalized LCIA method

In [9]:
data = {
    'code': 'freshwater',
    'name': 'freshwater consumption',
    'location': 'GLO',
    'unit': 'm3',
    'type': 'stressor'
}

freshwater = db.new_activity(**data)
freshwater.save()

## Import regionalized map

Normally we would use the function `br.import_regionalized_cfs`, but the provided maps has regions where the CF is `Null`. This will break things, so we need to do this import ourselves. Luckily, it isn't too difficult.

### TODO: Should also add a global CF here

In [10]:
def import_regionalized_cfs_with_geopandas(geocollection, method_tuple, mapping, scaling_factor=1, global_cfs=None):
    assert (geocollection in br.geocollections 
            and br.geocollections[geocollection].get('kind') == 'vector'
            and "field" in br.geocollections[geocollection]
           )
    gdf = gp.read_file(br.geocollections[geocollection]['filepath'])
    id_label = br.geocollections[geocollection]["field"]
    
    method = bd.Method(method_tuple)
    method.metadata['geocollections'] = [geocollection]
    bd.methods.flush()
    
    data = []
    if global_cfs:
        data.extend(global_cfs)

    for index, feature in gdf.iterrows():
        for field_label, biosphere_flows in mapping.items():
            if feature[field_label] is None or np.isnan(feature[field_label]):
                continue
            else:
                for flow in biosphere_flows:
                    data.append((
                        flow, 
                        float(feature[field_label]) * scaling_factor, 
                        (geocollection, feature[id_label])
                    ))
    
    method.write(data)

In [11]:
bd.Method(("water consumption", "watershed scale")).register(unit="PDF")
import_regionalized_cfs_with_geopandas(
    "watersheds", 
    ("water consumption", "watershed scale"), 
    {'EF_margina': [freshwater.key]}
)

In [12]:
bd.Method(("water consumption", "watershed scale")).load()[:20]

[(('case', 'freshwater'), 6.243519e-09, ('watersheds', 54)),
 (('case', 'freshwater'), 7.974224e-09, ('watersheds', 35)),
 (('case', 'freshwater'), 3.777942e-09, ('watersheds', 49)),
 (('case', 'freshwater'), 7.909293e-09, ('watersheds', 73)),
 (('case', 'freshwater'), 4.475176e-09, ('watersheds', 112)),
 (('case', 'freshwater'), 1.6187929e-08, ('watersheds', 155)),
 (('case', 'freshwater'), 1.2216529e-08, ('watersheds', 2832)),
 (('case', 'freshwater'), 1.2816316e-08, ('watersheds', 2963)),
 (('case', 'freshwater'), 2.567833e-09, ('watersheds', 5638)),
 (('case', 'freshwater'), 2.291942e-09, ('watersheds', 5708)),
 (('case', 'freshwater'), 3.430738e-09, ('watersheds', 15425)),
 (('case', 'freshwater'), 1.55811e-10, ('watersheds', 23897)),
 (('case', 'freshwater'), 2.7739e-10, ('watersheds', 23844)),
 (('case', 'freshwater'), 6.08337e-10, ('watersheds', 23808)),
 (('case', 'freshwater'), 1.886911e-09, ('watersheds', 352)),
 (('case', 'freshwater'), 4.51159e-10, ('watersheds', 301)),
 (

# Calculate intersection

We can just use geopandas instead of the pandarus library.

In [13]:
def write_intersection(first, second, overwrite=False):
    if (first, second) in br.intersections and not overwrite:
        raise ValueError("Intersection already exists")
    
    for gc in (first, second):
        assert (gc in br.geocollections 
            and br.geocollections[gc].get('kind') == 'vector'
            and "field" in br.geocollections[gc]
           )
    assert br.geocollections[first]['filepath'] != br.geocollections[second]['filepath']
    
    df1 = gp.read_file(br.geocollections[first]['filepath'])
    df2 = gp.read_file(br.geocollections[second]['filepath'])
    id1 = br.geocollections[first]["field"]
    id2 = br.geocollections[second]["field"]

    assert id1 != id2, "Conflicting ID labels"
    
    intersection = gp.overlay(df1, df2)
    areas = intersection.to_crs("esri:54009").area  # World Mollweidge in square meters
    
    data = []
    for index, feature in intersection.iterrows():
        data.append(((first, feature[id1]), (second, feature[id2]), areas[index]))
    
    obj = br.Intersection((first, second))
    obj.register()
    obj.write(data)
    obj.create_reversed_intersection()

In [14]:
write_intersection("regions", "watersheds")

In [15]:
br.Intersection(("regions", "watersheds")).load()[:20]

[(('regions', 'CN-62'), ('watersheds', 10944), 587710246.891498),
 (('regions', 'CN-62'), ('watersheds', 10784), 2813916857.421469),
 (('regions', 'CN-62'), ('watersheds', 10976), 5624540.639780564),
 (('regions', 'CN-62'), ('watersheds', 10781), 4505834025.288664),
 (('regions', 'CN-62'), ('watersheds', 10945), 7706500913.38276),
 (('regions', 'CN-62'), ('watersheds', 10979), 6601614202.278695),
 (('regions', 'CN-62'), ('watersheds', 10949), 913736681.8139017),
 (('regions', 'CN-62'), ('watersheds', 11263), 5083478143.000593),
 (('regions', 'CN-62'), ('watersheds', 11347), 149965830.59423748),
 (('regions', 'CN-62'), ('watersheds', 11344), 7290229563.897603),
 (('regions', 'CN-62'), ('watersheds', 11349), 71420084.42502172),
 (('regions', 'CN-62'), ('watersheds', 12236), 573727299.6146897),
 (('regions', 'CN-62'), ('watersheds', 12359), 2245344177.179704),
 (('regions', 'CN-62'), ('watersheds', 11641), 273397943.06686234),
 (('regions', 'CN-62'), ('watersheds', 11627), 420060145.33159

# Setup inventory graph

In [16]:
data = {
    'code': 'rice india',
    'name': 'rice production india',
    'location': ('regions','IN-MP'),
    'unit': 't'
}

rice_IN = db.new_activity(**data)
rice_IN.save()

In [17]:
data = {
    'code': 'rice china',
    'name': 'rice production china',
    'location': ('regions','CN-62'),
    'unit': 't'
}

rice_CN = db.new_activity(**data)
rice_CN.save()

In [18]:
data = {
    'code': 'rice USA',
    'name': 'rice production USA',
    'location': ('regions','US-MI'),
    'unit': 't'
}

rice_US = db.new_activity(**data)
rice_US.save()

In [19]:
data = {
    'code': 'pesticide',
    'name': 'pesticide production',
    'location': 'GLO',
    'unit': 't'
}

pesticide = db.new_activity(**data)
pesticide.save()

In [20]:
rice_IN.new_exchange(#modified
    amount=100, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='biosphere',
    input=freshwater,
).save()

In [21]:
rice_CN.new_exchange(#modified
    amount=100, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='biosphere',
    input=freshwater,
).save()

In [22]:
rice_US.new_exchange(#modified
    amount=100, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='biosphere',
    input=freshwater,
).save()

In [23]:
pesticide.new_exchange(#modified
    amount=100, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='biosphere',
    input=freshwater,
).save()

In [24]:
rice_IN.new_exchange(#modified
    amount=1,
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='technosphere',
    input=pesticide,
).save()

In [25]:
rice_CN.new_exchange(#modified
    amount=1, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='technosphere',
    input=pesticide,
).save()

In [26]:
rice_US.new_exchange(#modified
    amount=1, 
    #uncertainty_type=2, 
    #minimum=,
    #maximum=, 
    type='technosphere',
    input=pesticide,
).save()

In [27]:
from bw2calc.lca import *

In [28]:
sg_lca = bc.LCA({rice_US: 1}, ("water consumption", "watershed scale")) # site generic
sg_lca.lci()
sg_lca.lcia()
sg_lca.score

0.0

In [29]:
lca = br.TwoSpatialScalesLCA({rice_IN: 1}, ("water consumption", "watershed scale"))

In [30]:
lca.lci()
lca.lcia()
lca.score

3.51488082756986e-10